In [18]:
import pandas as pd
import torch
from ResNet import res_net_101, res_net_50
import os
from torchvision.transforms import v2 as v2
from PIL import Image

In [10]:
net_50 = res_net_50(num_classes=7)
net_50_lw = res_net_50(num_classes=7, block_input_layout=(32,64,128,256))
net_101 = res_net_101(num_classes=7, block_input_layout=(32,64,128,256))

In [11]:
net_101_path = os.path.join(os.getcwd(), "models", "net_101_lw.pth")
net101_state_dict = torch.load(net_101_path, weights_only=False)

In [13]:
net_101.load_state_dict(net101_state_dict)

<All keys matched successfully>

In [15]:
net_101 = net_101.to("cuda")

In [20]:
test_transforms = v2.Compose([
    v2.Resize(size=(196, 196)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)
])

In [36]:
image_path = os.path.join(os.getcwd(), "test_img", "image_4.jpg")
image = Image.open(image_path)
image_tensor = test_transforms(image).unsqueeze(0).to("cuda")

In [37]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

sc_classes = [cls for cls in lesion_type_dict.items()]

In [39]:
net_101.eval()
with torch.inference_mode():
    logits = net_101(image_tensor) 
    probabilities = torch.softmax(logits, dim=1)
    top3_probs, top3_idx = torch.topk(probabilities, 3, dim=1)


top3_labels = [sc_classes[idx] for idx in top3_idx[0]]

# Print the predicted label and probability
for i in range(3):
    print(f'Prediction {i+1}: {top3_labels[i][1]} with probability {top3_probs[0][i].item():.4f}')

Prediction 1: Actinic keratoses with probability 0.7978
Prediction 2: Vascular lesions with probability 0.0885
Prediction 3: Benign keratosis-like lesions  with probability 0.0492
